<a href="https://colab.research.google.com/github/wildautumnwind/ml_notebooks/blob/master/homework_wine_titanic_knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Homework 1

Light: для DataSet Wine найти оптимальные параметры для метода kNN

Pro: для DataSet Titanic найти оптимальные параметры для метод kNN

https://www.kaggle.com/c/titanic/data

# Load libs

In [0]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
import sklearn
import numpy as np
from google.colab import files
import pandas as pd
import matplotlib.pyplot as plt

# User defined functions

In [0]:
# Select best metrics from the given
def select_best_metrics(X_train, y_train, X, y):
  best_metrics_selected = {}

  metrics = ['euclidean', 'manhattan', 'minkowski']
  weights = ['uniform', 'distance']
  k_nums = range(2, 100)
  best_accuracy = 0.0

  for weight in weights:
    for metric in metrics:
      for k_num in k_nums:  
        model = KNeighborsClassifier(n_neighbors = k_num, metric = metric, weights = weight)
        model.fit(X_train, y_train)

        y_pred = model.predict(X)
        current_accuracy = accuracy_score(y, y_pred)
        if current_accuracy > best_accuracy:
          best_accuracy = current_accuracy
          best_metrics_selected['metric'] = metric
          best_metrics_selected['n_neighbors'] = k_num
          best_metrics_selected['weights'] = weight

  return best_metrics_selected

# Light (wine dataset)

## Load dataset

In [0]:
# load dataset
from sklearn.datasets import load_wine
data_wine = load_wine()

In [3]:
# dataset description
print(data_wine.DESCR)

.. _wine_dataset:

Wine recognition dataset
------------------------

**Data Set Characteristics:**

    :Number of Instances: 178 (50 in each of three classes)
    :Number of Attributes: 13 numeric, predictive attributes and the class
    :Attribute Information:
 		- Alcohol
 		- Malic acid
 		- Ash
		- Alcalinity of ash  
 		- Magnesium
		- Total phenols
 		- Flavanoids
 		- Nonflavanoid phenols
 		- Proanthocyanins
		- Color intensity
 		- Hue
 		- OD280/OD315 of diluted wines
 		- Proline

    - class:
            - class_0
            - class_1
            - class_2
		
    :Summary Statistics:
    
    ============================= ==== ===== ======= =====
                                   Min   Max   Mean     SD
    ============================= ==== ===== ======= =====
    Alcohol:                      11.0  14.8    13.0   0.8
    Malic Acid:                   0.74  5.80    2.34  1.12
    Ash:                          1.36  3.23    2.36  0.27
    Alcalinity of Ash:            1

In [4]:
data_wine.data.shape

(178, 13)

## Data exploration

In [5]:
# Data targets (classes)
data_wine.target

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2])

In [6]:
# Target names (names of classes)
data_wine.target_names

array(['class_0', 'class_1', 'class_2'], dtype='<U7')

In [7]:
# Features names
data_wine.feature_names

['alcohol',
 'malic_acid',
 'ash',
 'alcalinity_of_ash',
 'magnesium',
 'total_phenols',
 'flavanoids',
 'nonflavanoid_phenols',
 'proanthocyanins',
 'color_intensity',
 'hue',
 'od280/od315_of_diluted_wines',
 'proline']

In [0]:
# Get data and labels for training
X = data_wine.data
y = data_wine.target

In [9]:
# Data exploration
print('Data: ' + str(X[:5]))
print("\n")
print('Data shape: ' + str(X.shape))

Data: [[1.423e+01 1.710e+00 2.430e+00 1.560e+01 1.270e+02 2.800e+00 3.060e+00
  2.800e-01 2.290e+00 5.640e+00 1.040e+00 3.920e+00 1.065e+03]
 [1.320e+01 1.780e+00 2.140e+00 1.120e+01 1.000e+02 2.650e+00 2.760e+00
  2.600e-01 1.280e+00 4.380e+00 1.050e+00 3.400e+00 1.050e+03]
 [1.316e+01 2.360e+00 2.670e+00 1.860e+01 1.010e+02 2.800e+00 3.240e+00
  3.000e-01 2.810e+00 5.680e+00 1.030e+00 3.170e+00 1.185e+03]
 [1.437e+01 1.950e+00 2.500e+00 1.680e+01 1.130e+02 3.850e+00 3.490e+00
  2.400e-01 2.180e+00 7.800e+00 8.600e-01 3.450e+00 1.480e+03]
 [1.324e+01 2.590e+00 2.870e+00 2.100e+01 1.180e+02 2.800e+00 2.690e+00
  3.900e-01 1.820e+00 4.320e+00 1.040e+00 2.930e+00 7.350e+02]]


Data shape: (178, 13)


In [10]:
# Labels exploration
print('Data: ' + str(y[:5]))
print("\n")
print('Data shape: ' + str(y.shape))

Data: [0 0 0 0 0]


Data shape: (178,)


## Training

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [14]:
print('Train data shape - ' + str(X_train.shape) + ', train labels shape - ' + str(y_train.shape))
print('Test data shape - ' + str(X_test.shape) + ', test labels shape - ' + str(y_test.shape))

Train data shape - (142, 13), train labels shape - (142,)
Test data shape - (36, 13), test labels shape - (36,)


In [24]:
# Check valid metrics
print(sklearn.neighbors.VALID_METRICS['brute'])

['cityblock', 'cosine', 'euclidean', 'haversine', 'l2', 'l1', 'manhattan', 'precomputed', 'braycurtis', 'canberra', 'chebyshev', 'correlation', 'cosine', 'dice', 'hamming', 'jaccard', 'kulsinski', 'mahalanobis', 'matching', 'minkowski', 'rogerstanimoto', 'russellrao', 'seuclidean', 'sokalmichener', 'sokalsneath', 'sqeuclidean', 'yule', 'wminkowski']


In [0]:
# Call udf to get best parameters
best_metrics = select_best_metrics(X_train, y_train, X, y)

In [80]:
# Check best parameters
best_metrics

{'metric': 'manhattan', 'n_neighbors': 15, 'weights': 'distance'}

In [62]:
# Fit model with best parameters selected previously
model = KNeighborsClassifier(n_neighbors = best_metrics['n_neighbors'], metric = best_metrics['metric'], weights = best_metrics['weights'])
model.fit(X_train,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='manhattan',
                     metric_params=None, n_jobs=None, n_neighbors=15, p=2,
                     weights='distance')

In [69]:
# Get predictions
y_pred = model.predict(X_test)
y_pred

array([0, 0, 2, 0, 1, 0, 1, 2, 2, 0, 0, 2, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1,
       2, 2, 1, 2, 1, 2, 1, 0, 0, 1, 2, 0, 0, 0])

In [65]:
print('Accuracy: ', accuracy_score(y_test, y_pred), accuracy_score(y_test, y_pred, normalize = False))

Accuracy:  0.8611111111111112 31


# Pro (titanic dataset)

## Load dataset

In [71]:
!pip install kaggle

In [72]:
file = files.upload()

Saving kaggle.json to kaggle.json


In [0]:
!mkdir ~/.kaggle
!mv kaggle.json ~/.kaggle

In [74]:
!kaggle competitions download -c titanic

  0% 0.00/59.8k [00:00<?, ?B/s]
100% 59.8k/59.8k [00:00<00:00, 61.0MB/s]
  0% 0.00/28.0k [00:00<?, ?B/s]
100% 28.0k/28.0k [00:00<00:00, 29.6MB/s]
  0% 0.00/3.18k [00:00<?, ?B/s]
100% 3.18k/3.18k [00:00<00:00, 2.83MB/s]


## Data exploration

In [0]:
test = pd.read_csv('test.csv')
train = pd.read_csv('train.csv')

In [76]:
# Test data (we will use it for prediction)
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [77]:
# Data for model training
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Training

In [0]:
y = train["Survived"]

In [0]:
features = ["Pclass", "Sex", "SibSp", "Parch"]
X = pd.get_dummies(train[features])

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [86]:
print('Train data shape - ' + str(X_train.shape) + ', train labels shape - ' + str(y_train.shape))
print('Test data shape - ' + str(X_test.shape) + ', test labels shape - ' + str(y_test.shape))

Train data shape - (712, 5), train labels shape - (712,)
Test data shape - (179, 5), test labels shape - (179,)


In [0]:
# Call udf to get best parameters
best_metrics = select_best_metrics(X_train, y_train, X, y)

In [88]:
# Check best parameters
best_metrics

{'metric': 'euclidean', 'n_neighbors': 5, 'weights': 'distance'}

In [89]:
# Fit model with best parameters selected previously
model = KNeighborsClassifier(n_neighbors = best_metrics['n_neighbors'], metric = best_metrics['metric'], weights = best_metrics['weights'])
model.fit(X_train,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='euclidean',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='distance')

In [90]:
# Get predictions
y_pred = model.predict(X_test)
y_pred

array([0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1,
       0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0,
       0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0,
       0, 1, 1])

In [91]:
print('Accuracy: ', accuracy_score(y_test, y_pred), accuracy_score(y_test, y_pred, normalize = False))

Accuracy:  0.7877094972067039 141


# Useful links

* https://kevinzakka.github.io/2016/07/13/k-nearest-neighbor/